In [1]:
#Traffic analysis 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import warnings

In [7]:
def clean_value(val):
    """Remove newlines and whitespace, convert to numeric if possible"""
    if pd.isna(val):
        return np.nan
    if isinstance(val, (int, float)):
        return val
    if isinstance(val, str):
        cleaned = val.replace('\n', '').strip()
        if cleaned == '':
            return np.nan
        try:
            return float(cleaned) if '.' in cleaned else int(cleaned)
        except ValueError:
            return np.nan
    return np.nan

def parse_datetime(val):
    """Parse the datetime string format: 'Sab - 01/02/2025 0:00 - 0:15'"""
    if pd.isna(val):
        return None, None, None
    cleaned = val.replace('\n', '').strip()
    parts = cleaned.split(' - ')
    if len(parts) >= 2:
        day_name = parts[0].strip()
        date_time = parts[1].strip()
        date_part = date_time.split(' ')[0]
        time_start = date_time.split(' ')[1] if len(date_time.split(' ')) > 1 else '0:00'
        try:
            dt = datetime.strptime(f"{date_part} {time_start}", "%d/%m/%Y %H:%M")
            return dt, day_name, time_start
        except:
            return None, day_name, None
    return None, None, None

def load_and_clean_data():
    """Load all three datasets and clean them"""
    print("=" * 70)
    print("DATA LOADING AND PREPROCESSING")
    print("=" * 70)
    
    # Load raw data
    classi = pd.read_excel('/Users/thomas/Data/7_AID_webcam_data/Febbraio/classi.xlsx')
    corsie = pd.read_excel('/Users/thomas/Data/7_AID_webcam_data/Febbraio/corsie.xlsx')
    transiti = pd.read_excel('/Users/thomas/Data/7_AID_webcam_data/Febbraio/transiti.xlsx')
    
    print(f"\nOriginal dataset sizes:")
    print(f"  - Classi (Vehicle Types): {classi.shape[0]} rows, {classi.shape[1]} columns")
    print(f"  - Corsie (Lanes): {corsie.shape[0]} rows, {corsie.shape[1]} columns")
    print(f"  - Transiti (Total): {transiti.shape[0]} rows, {transiti.shape[1]} columns")
    
    # Clean CLASSI dataset
    classi_clean = classi.copy()
    for col in ['Car', 'Bus', 'Motorbike', 'Truck', 'Van']:
        classi_clean[col] = classi_clean[col].apply(clean_value)
    
    # Parse datetime for classi
    classi_clean['datetime'] = classi_clean['Data e ora'].apply(lambda x: parse_datetime(x)[0])
    classi_clean['day_name'] = classi_clean['Data e ora'].apply(lambda x: parse_datetime(x)[1])
    classi_clean['time'] = classi_clean['datetime'].apply(lambda x: x.time() if x else None)
    classi_clean['hour'] = classi_clean['datetime'].apply(lambda x: x.hour if x else None)
    classi_clean['date'] = classi_clean['datetime'].apply(lambda x: x.date() if x else None)
    
    # Clean CORSIE dataset
    corsie_clean = corsie.copy()
    for col in ['corsia1', 'emergenza', 'corsia2', 'corsia3']:
        corsie_clean[col] = corsie_clean[col].apply(clean_value)
    
    corsie_clean['datetime'] = corsie_clean['Data e ora'].apply(lambda x: parse_datetime(x)[0])
    corsie_clean['day_name'] = corsie_clean['Data e ora'].apply(lambda x: parse_datetime(x)[1])
    corsie_clean['hour'] = corsie_clean['datetime'].apply(lambda x: x.hour if x else None)
    corsie_clean['date'] = corsie_clean['datetime'].apply(lambda x: x.date() if x else None)
    corsie_clean['total_lanes'] = corsie_clean['corsia1'] + corsie_clean['corsia2'] + corsie_clean['corsia3'] + corsie_clean['emergenza']
    
    # Clean TRANSITI dataset  
    transiti_clean = transiti.copy()
    transiti_clean['total_vehicles'] = transiti_clean['A7_41+550_N_AID_N'].apply(clean_value)
    transiti_clean['datetime'] = transiti_clean['Data e ora'].apply(lambda x: parse_datetime(x)[0])
    transiti_clean['day_name'] = transiti_clean['Data e ora'].apply(lambda x: parse_datetime(x)[1])
    transiti_clean['hour'] = transiti_clean['datetime'].apply(lambda x: x.hour if x else None)
    transiti_clean['date'] = transiti_clean['datetime'].apply(lambda x: x.date() if x else None)
    
    # Add total to classi for cross-validation
    for col in ['Car', 'Bus', 'Motorbike', 'Truck', 'Van']:
        classi_clean[col] = pd.to_numeric(classi_clean[col], errors='coerce').fillna(0).astype(int)
    classi_clean['total_classi'] = classi_clean['Car'] + classi_clean['Bus'] + classi_clean['Motorbike'] + classi_clean['Truck'] + classi_clean['Van']
    
    # Ensure corsie columns are numeric
    for col in ['corsia1', 'emergenza', 'corsia2', 'corsia3']:
        corsie_clean[col] = pd.to_numeric(corsie_clean[col], errors='coerce').fillna(0).astype(int)
    
    # Ensure transiti is numeric
    transiti_clean['total_vehicles'] = pd.to_numeric(transiti_clean['total_vehicles'], errors='coerce').fillna(0).astype(int)
    
    return classi_clean, corsie_clean, transiti_clean

def check_data_quality(classi, corsie, transiti):
    """Check for duplicates, missing values, and cross-validate"""
    print("\n" + "=" * 70)
    print("DATA QUALITY ASSESSMENT")
    print("=" * 70)
    
    # Missing values count
    print("\n--- Missing Values Count ---")
    print("\nClassi (Vehicle Types):")
    missing_classi = classi[['Car', 'Bus', 'Motorbike', 'Truck', 'Van', 'datetime']].isnull().sum()
    print(missing_classi)
    
    print("\nCorsie (Lanes):")
    missing_corsie = corsie[['corsia1', 'emergenza', 'corsia2', 'corsia3', 'datetime']].isnull().sum()
    print(missing_corsie)
    
    print("\nTransiti (Total):")
    missing_transiti = transiti[['total_vehicles', 'datetime']].isnull().sum()
    print(missing_transiti)
    
    # Duplicates
    print("\n--- Duplicate Rows ---")
    dup_classi = classi.duplicated(subset=['datetime']).sum()
    dup_corsie = corsie.duplicated(subset=['datetime']).sum()
    dup_transiti = transiti.duplicated(subset=['datetime']).sum()
    print(f"Classi duplicates (by datetime): {dup_classi}")
    print(f"Corsie duplicates (by datetime): {dup_corsie}")
    print(f"Transiti duplicates (by datetime): {dup_transiti}")
    
    # Remove duplicates if any
    if dup_classi > 0:
        classi = classi.drop_duplicates(subset=['datetime'], keep='first')
    if dup_corsie > 0:
        corsie = corsie.drop_duplicates(subset=['datetime'], keep='first')
    if dup_transiti > 0:
        transiti = transiti.drop_duplicates(subset=['datetime'], keep='first')
    
    # Cross-validation of totals
    print("\n--- Cross-Validation of Vehicle Counts ---")
    
    # Merge datasets on datetime for comparison
    merged = classi[['datetime', 'total_classi']].merge(
        corsie[['datetime', 'total_lanes']], on='datetime', how='outer'
    ).merge(
        transiti[['datetime', 'total_vehicles']], on='datetime', how='outer'
    )
    
    # Compare totals
    merged['classi_vs_transiti'] = merged['total_classi'] - merged['total_vehicles']
    merged['lanes_vs_transiti'] = merged['total_lanes'] - merged['total_vehicles']
    
    print(f"\nTotal records after merge: {len(merged)}")
    print(f"Records with matching totals (classi vs transiti): {(merged['classi_vs_transiti'] == 0).sum()}")
    print(f"Records with matching totals (lanes vs transiti): {(merged['lanes_vs_transiti'] == 0).sum()}")
    
    # Summary statistics of discrepancies
    print(f"\nDiscrepancy statistics (classi - transiti):")
    print(f"  Mean: {merged['classi_vs_transiti'].mean():.2f}")
    print(f"  Std: {merged['classi_vs_transiti'].std():.2f}")
    print(f"  Max: {merged['classi_vs_transiti'].max():.0f}")
    print(f"  Min: {merged['classi_vs_transiti'].min():.0f}")
    
    return classi, corsie, transiti, merged

def add_temporal_features(df):
    """Add weekday/weekend and other temporal features"""
    df = df.copy()
    
    # Map Italian day names to day of week
    day_map = {'Lun': 0, 'Mar': 1, 'Mer': 2, 'Gio': 3, 'Ven': 4, 'Sab': 5, 'Dom': 6}
    df['day_of_week'] = df['day_name'].map(day_map)
    
    # Weekday vs Weekend
    df['is_weekend'] = df['day_of_week'].isin([5, 6])
    df['day_type'] = df['is_weekend'].map({True: 'Weekend', False: 'Weekday'})
    
    # Time of day categories
    def categorize_time(hour):
        if hour is None:
            return None
        if 6 <= hour < 10:
            return 'Morning Rush (6-10)'
        elif 10 <= hour < 16:
            return 'Midday (10-16)'
        elif 16 <= hour < 20:
            return 'Evening Rush (16-20)'
        else:
            return 'Night (20-6)'
    
    df['time_category'] = df['hour'].apply(categorize_time)
    
    return df

# ============================================================
# ANALYSIS FUNCTIONS
# ============================================================

def analyze_overall_traffic(classi, transiti):
    """Overall traffic statistics"""
    print("\n" + "=" * 70)
    print("OVERALL TRAFFIC STATISTICS")
    print("=" * 70)
    
    # Daily totals
    daily_traffic = transiti.groupby('date')['total_vehicles'].sum()
    
    print(f"\nTotal vehicles in February 2025: {transiti['total_vehicles'].sum():,.0f}")
    print(f"Average daily traffic: {daily_traffic.mean():,.0f}")
    print(f"Minimum daily traffic: {daily_traffic.min():,.0f} (on {daily_traffic.idxmin()})")
    print(f"Maximum daily traffic: {daily_traffic.max():,.0f} (on {daily_traffic.idxmax()})")
    
    # Per 15-min interval stats
    print(f"\nPer 15-minute interval statistics:")
    print(f"  Average: {transiti['total_vehicles'].mean():.1f} vehicles")
    print(f"  Maximum: {transiti['total_vehicles'].max():.0f} vehicles")
    print(f"  Minimum: {transiti['total_vehicles'].min():.0f} vehicles")
    
    return daily_traffic

def analyze_vehicle_classification(classi):
    """Analyze vehicle type distribution"""
    print("\n" + "=" * 70)
    print("VEHICLE CLASSIFICATION ANALYSIS")
    print("=" * 70)
    
    vehicle_types = ['Car', 'Bus', 'Motorbike', 'Truck', 'Van']
    totals = {vtype: classi[vtype].sum() for vtype in vehicle_types}
    grand_total = sum(totals.values())
    
    print("\nVehicle Type Distribution:")
    print("-" * 40)
    for vtype, count in sorted(totals.items(), key=lambda x: x[1], reverse=True):
        pct = (count / grand_total) * 100
        print(f"  {vtype:12s}: {count:>10,.0f} ({pct:5.1f}%)")
    print("-" * 40)
    print(f"  {'Total':12s}: {grand_total:>10,.0f}")
    
    # Heavy vehicles analysis
    heavy_vehicles = totals['Truck'] + totals['Bus']
    heavy_pct = (heavy_vehicles / grand_total) * 100
    print(f"\nHeavy Vehicles (Trucks + Buses): {heavy_vehicles:,.0f} ({heavy_pct:.1f}%)")
    
    return totals

def analyze_heavy_vehicle_patterns(classi):
    """Detailed analysis of heavy vehicle patterns"""
    print("\n" + "=" * 70)
    print("HEAVY VEHICLE PATTERN ANALYSIS")
    print("=" * 70)
    
    classi['heavy_vehicles'] = classi['Truck'] + classi['Bus']
    
    # Weekday vs Weekend patterns
    heavy_by_daytype = classi.groupby('day_type')['heavy_vehicles'].agg(['sum', 'mean'])
    print("\nHeavy Vehicles by Day Type:")
    print(heavy_by_daytype)
    
    # Hourly patterns
    heavy_by_hour = classi.groupby(['day_type', 'hour'])['heavy_vehicles'].mean().unstack(level=0)
    
    print("\nPeak Heavy Vehicle Hours (Weekday):")
    weekday_heavy = classi[classi['day_type'] == 'Weekday'].groupby('hour')['heavy_vehicles'].mean()
    top_hours = weekday_heavy.nlargest(5)
    for hour, count in top_hours.items():
        print(f"  {hour:02d}:00 - {hour:02d}:59: {count:.1f} avg heavy vehicles per 15min")
    
    # Truck vs Bus breakdown
    print("\nTruck vs Bus Analysis:")
    truck_by_daytype = classi.groupby('day_type')['Truck'].sum()
    bus_by_daytype = classi.groupby('day_type')['Bus'].sum()
    print(f"  Trucks - Weekday: {truck_by_daytype.get('Weekday', 0):,.0f}, Weekend: {truck_by_daytype.get('Weekend', 0):,.0f}")
    print(f"  Buses - Weekday: {bus_by_daytype.get('Weekday', 0):,.0f}, Weekend: {bus_by_daytype.get('Weekend', 0):,.0f}")
    
    # Truck ratio (proportion of heavy vehicles that are trucks)
    truck_ratio_weekday = truck_by_daytype.get('Weekday', 0) / (truck_by_daytype.get('Weekday', 0) + bus_by_daytype.get('Weekday', 0))
    truck_ratio_weekend = truck_by_daytype.get('Weekend', 0) / (truck_by_daytype.get('Weekend', 0) + bus_by_daytype.get('Weekend', 0))
    print(f"\nTruck as % of heavy vehicles:")
    print(f"  Weekday: {truck_ratio_weekday*100:.1f}%")
    print(f"  Weekend: {truck_ratio_weekend*100:.1f}%")
    
    return heavy_by_hour

def analyze_peak_hours(classi, transiti):
    """Identify peak traffic hours"""
    print("\n" + "=" * 70)
    print("PEAK HOUR ANALYSIS")
    print("=" * 70)
    
    # Overall hourly patterns
    hourly_total = transiti.groupby(['day_type', 'hour'])['total_vehicles'].mean()
    
    # Weekday peaks
    weekday_hourly = hourly_total.xs('Weekday', level=0)
    print("\nWeekday Peak Hours (top 5):")
    for hour, count in weekday_hourly.nlargest(5).items():
        print(f"  {hour:02d}:00 - {hour:02d}:59: {count:.1f} avg vehicles per 15min")
    
    # Weekend peaks
    weekend_hourly = hourly_total.xs('Weekend', level=0)
    print("\nWeekend Peak Hours (top 5):")
    for hour, count in weekend_hourly.nlargest(5).items():
        print(f"  {hour:02d}:00 - {hour:02d}:59: {count:.1f} avg vehicles per 15min")
    
    # Morning vs Evening rush
    weekday_data = transiti[transiti['day_type'] == 'Weekday']
    morning_rush = weekday_data[weekday_data['hour'].between(6, 9)]['total_vehicles'].mean()
    evening_rush = weekday_data[weekday_data['hour'].between(16, 19)]['total_vehicles'].mean()
    
    print(f"\nWeekday Rush Hour Comparison:")
    print(f"  Morning Rush (6-10): {morning_rush:.1f} avg vehicles per 15min")
    print(f"  Evening Rush (16-20): {evening_rush:.1f} avg vehicles per 15min")
    
    return hourly_total

def analyze_lane_distribution(corsie):
    """Analyze traffic distribution across lanes"""
    print("\n" + "=" * 70)
    print("LANE DISTRIBUTION ANALYSIS")
    print("=" * 70)
    
    lanes = ['corsia1', 'corsia2', 'corsia3', 'emergenza']
    lane_totals = {lane: corsie[lane].sum() for lane in lanes}
    grand_total = sum(lane_totals.values())
    
    print("\nTraffic Distribution by Lane:")
    print("-" * 40)
    for lane, count in sorted(lane_totals.items(), key=lambda x: x[1], reverse=True):
        pct = (count / grand_total) * 100
        print(f"  {lane:12s}: {count:>10,.0f} ({pct:5.1f}%)")
    
    # Lane usage by time of day
    print("\nLane Usage by Time Category:")
    lane_by_time = corsie.groupby('time_category')[lanes].sum()
    print(lane_by_time)
    
    return lane_totals


In [13]:
classi, corsie, transiti = load_and_clean_data()


print(corsie.tail())

DATA LOADING AND PREPROCESSING

Original dataset sizes:
  - Classi (Vehicle Types): 2688 rows, 6 columns
  - Corsie (Lanes): 2688 rows, 5 columns
  - Transiti (Total): 2688 rows, 2 columns
                                             Data e ora  corsia1  emergenza  \
2683                 \nVen - 28/02/2025 22:45 - 23:00\n       28          0   
2684                 \nVen - 28/02/2025 23:00 - 23:15\n       37          0   
2685                 \nVen - 28/02/2025 23:15 - 23:30\n       29          0   
2686                 \nVen - 28/02/2025 23:30 - 23:45\n       42          0   
2687  \nVen - 28/02/2025 23:45 - Sab - 01/03/2025 0:...       35          0   

      corsia2  corsia3            datetime day_name  hour        date  \
2683       21        2 2025-02-28 22:45:00      Ven    22  2025-02-28   
2684       35        4 2025-02-28 23:00:00      Ven    23  2025-02-28   
2685       44        4 2025-02-28 23:15:00      Ven    23  2025-02-28   
2686       27        4 2025-02-28 23:30:00  

In [14]:
classi, corsie, transiti, merged = check_data_quality(classi, corsie, transiti)

print(classi.head())


DATA QUALITY ASSESSMENT

--- Missing Values Count ---

Classi (Vehicle Types):
Car          0
Bus          0
Motorbike    0
Truck        0
Van          0
datetime     0
dtype: int64

Corsie (Lanes):
corsia1      0
emergenza    0
corsia2      0
corsia3      0
datetime     0
dtype: int64

Transiti (Total):
total_vehicles    0
datetime          0
dtype: int64

--- Duplicate Rows ---
Classi duplicates (by datetime): 0
Corsie duplicates (by datetime): 0
Transiti duplicates (by datetime): 0

--- Cross-Validation of Vehicle Counts ---

Total records after merge: 2688
Records with matching totals (classi vs transiti): 2660
Records with matching totals (lanes vs transiti): 2587

Discrepancy statistics (classi - transiti):
  Mean: -0.01
  Std: 0.16
  Max: 0
  Min: -3
                         Data e ora  Car  Bus  Motorbike  Truck  Van  \
0  \nSab - 01/02/2025 0:00 - 0:15\n   24    4          0      3    7   
1  \nSab - 01/02/2025 0:15 - 0:30\n   38    3          0      6   12   
2  \nSab - 01/0

In [17]:
classi = add_temporal_features(classi)
corsie = add_temporal_features(corsie)
transiti = add_temporal_features(transiti)

print(classi.tail(20))

                                             Data e ora  Car  Bus  Motorbike  \
2668                 \nVen - 28/02/2025 19:00 - 19:15\n  281   15          2   
2669                 \nVen - 28/02/2025 19:15 - 19:30\n  268    8          0   
2670                 \nVen - 28/02/2025 19:30 - 19:45\n  286   13          0   
2671                 \nVen - 28/02/2025 19:45 - 20:00\n  230   11          0   
2672                 \nVen - 28/02/2025 20:00 - 20:15\n  164   10          0   
2673                 \nVen - 28/02/2025 20:15 - 20:30\n  156   11          1   
2674                 \nVen - 28/02/2025 20:30 - 20:45\n  203   13          0   
2675                 \nVen - 28/02/2025 20:45 - 21:00\n  173   20          0   
2676                 \nVen - 28/02/2025 21:00 - 21:15\n  128   24          0   
2677                 \nVen - 28/02/2025 21:15 - 21:30\n   97   18          0   
2678                 \nVen - 28/02/2025 21:30 - 21:45\n   67   13          0   
2679                 \nVen - 28/02/2025 

In [20]:
daily_traffic = analyze_overall_traffic(classi, transiti)
vehicle_totals = analyze_vehicle_classification(classi)
heavy_patterns = analyze_heavy_vehicle_patterns(classi)
peak_hours = analyze_peak_hours(classi, transiti)
lane_distribution = analyze_lane_distribution(corsie)


#print(vehicle_totals.head())


OVERALL TRAFFIC STATISTICS

Total vehicles in February 2025: 619,930
Average daily traffic: 22,140
Minimum daily traffic: 8,076 (on 2025-02-17)
Maximum daily traffic: 30,637 (on 2025-02-16)

Per 15-minute interval statistics:
  Average: 230.6 vehicles
  Maximum: 851 vehicles
  Minimum: 0 vehicles

VEHICLE CLASSIFICATION ANALYSIS

Vehicle Type Distribution:
----------------------------------------
  Car         :    424,283 ( 68.4%)
  Truck       :    107,132 ( 17.3%)
  Van         :     70,648 ( 11.4%)
  Bus         :     16,906 (  2.7%)
  Motorbike   :        921 (  0.1%)
----------------------------------------
  Total       :    619,890

Heavy Vehicles (Trucks + Buses): 124,038 (20.0%)

HEAVY VEHICLE PATTERN ANALYSIS

Heavy Vehicles by Day Type:
             sum       mean
day_type                   
Weekday   114072  59.412500
Weekend     9966  12.976562

Peak Heavy Vehicle Hours (Weekday):
  06:00 - 06:59: 155.7 avg heavy vehicles per 15min
  10:00 - 10:59: 116.2 avg heavy vehicl